In [ ]:
from scipy.stats import skewnorm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


def simulate_stock(initial_price, drift, volatility, trend, days):

  def create_pdf(sd, mean, alfa):
    #invertire il segno di alfa
    x = skewnorm.rvs(alfa, size=1000000) 
    def calc(k, sd, mean):
      return (k*sd)+mean
    x = calc(x, sd, mean) #standard distribution

    #graph pdf
    #pd.DataFrame(x).hist(bins=100)

    #pick one random number from the distribution
    #formally I would use cdf, but I just have to pick randomly from the 1000000 samples
    #np.random.choice(x)
    return x

  def create_empty_df(days):
    #creare un empty DataFrame con le date
    empty = pd.Series(
        pd.date_range("2019-01-01", periods=days, freq="D")
    )
    empty = pd.DataFrame(empty)
    #si tagliano ore, minuti, secondi
    empty

    #si tagliano ore, minuti, secondi
    empty.index = [str(x)[0:empty.shape[0]] for x in list(empty.pop(0))]
    empty

    #final dataset con values
    stock = pd.DataFrame([x for x in range(0, empty.shape[0])])
    stock.index = empty.index
    return stock

  #skeleton
  stock = create_empty_df(days)

  #initial price
  stock[0][0] = initial_price

  #create entire stock DataFrame
  x = create_pdf(volatility, drift, trend)
  for _ in range(1, stock.shape[0]):
    stock.iloc[_] = stock.iloc[_-1]*(1+np.random.choice(x))

  return stock

In [ ]:
def graph_stocks(stock):
  #graph all stocks
  plt.figure(figsize=(28, 14))
  for c in stock.columns.values:
    #print(c)
    fig = plt.plot(stock.index, stock[c], lw=1, alpha=1)
  #plt.legend(loc='upper left', fontsize=12)
  #plt.ylabel('price in $')

  #remove axis labels
  plt.gca().axes.get_yaxis().set_visible(False)

In [ ]:
#create multiple stocks
stock_list = list()
column_name = list()

#creating a benchmark stock
column_name.append('benchmark')
stock_list.append(simulate_stock(1000, 0, 0.0, 0, 100))

n_iter = 1
n_days = 100

for _ in range(n_iter):
  #naming the stocks
  text = 'stock, '
  column_name.append(text + str(_))

  #creating the simulated stock
  stock_list.append(simulate_stock(1000, 0.00, 0.02, 0, n_days))

#join the stocks together
stock = pd.concat([x for x in stock_list], axis=1)
stock.columns = column_name

stock

In [ ]:
#create multiple stocks
stock_list = list()

column_name = list()

#benchmark
column_name.append('benchmark')
stock_list.append(simulate_stock(1000, 0, 0.0, 0, 100))

n_iter = 100
n_days = 100

for _ in range(n_iter):
  text = '-0.00, 0.03, '
  column_name.append(text + str(_))
  stock_list.append(simulate_stock(1000, -0.01, 0.01, 0, n_days))

for _ in range(n_iter):
  text = '-0.00, 0.03, '
  column_name.append(text + str(_))
  stock_list.append(simulate_stock(1000, -0.01, 0.01, 0, n_days))

for _ in range(n_iter):
  text = '-0.01, 0.01, '
  column_name.append(text + str(_))
  stock_list.append(simulate_stock(1000, -0.01, 0.01, 0, n_days))

for _ in range(n_iter):
  text = '+0.01, 0.01, '
  column_name.append(text + str(_))
  stock_list.append(simulate_stock(1000, +0.01, 0.01, 0, n_days))

for _ in range(n_iter):
  text = '-0.01, 0.03, '
  column_name.append(text + str(_))
  stock_list.append(simulate_stock(1000, -0.01, 0.03, 0, n_days))

for _ in range(n_iter):
  text = '+0.01, 0.03, '
  column_name.append(text + str(_))
  stock_list.append(simulate_stock(1000, +0.01, 0.03, 0, n_days))

for _ in range(n_iter):
  text = '-0.01, 0.05, '
  column_name.append(text + str(_))
  stock_list.append(simulate_stock(1000, -0.01, 0.05, 0, n_days))

for _ in range(n_iter):
  text = '+0.01, 0.05, '
  column_name.append(text + str(_))
  stock_list.append(simulate_stock(1000, +0.01, 0.05, 0, n_days))

for _ in range(n_iter):
  text = '-0.01, 0.10, '
  column_name.append(text + str(_))
  stock_list.append(simulate_stock(1000, -0.01, 0.10, 0, n_days))

for _ in range(n_iter):
  text = '+0.01, 0.10, '
  column_name.append(text + str(_))
  stock_list.append(simulate_stock(1000, +0.01, 0.10, 0, n_days))


#join the stocks together
stock = pd.concat([x for x in stock_list], axis=1)
stock.columns = column_name

#graph_stocks(stock)

#mean #positive, negative, h. positive, h. negative     #sd 0.01, 0.03, 0.05, 0.10
#12000

stock

In [ ]:
graph_stocks(stock)

In [ ]:
stock.to_csv('stock.csv')

##Analysis

In [ ]:
graph_stocks(stock)

In [ ]:
stock.pct_change().dropna()

In [ ]:
delta = stock.diff(1).dropna()
delta

up = delta.copy()
down = delta.copy()

up[up<0]=0
up

down[down>0]=0
down

In [ ]:
period = 3

AVG_gain = up.rolling(window=period).mean()
AVG_loss = abs(down.rolling(window=period).mean())

In [ ]:
AVG_gain

In [ ]:
AVG_loss

In [ ]:
#Relative Strenght
RS = AVG_gain/AVG_loss

#RSI
RSI = 100.0 - (100.0 / (1.0 + RS))
RSI

In [ ]:
#plt.figure(figsize=(12.2, 4.5))
RSI.plot()
plt.gcf().set_size_inches(20, 10)
plt.show()

In [ ]:
graph_stocks(pd.concat([stock, RSI], axis=1))